## PLANNING

This notebook describes the planning.py module, which covers Classical Planning from Chapter 10 & Planning & Acting in Real World Chapter 11.

In [1]:
from planning import *
from utils import expr

#PDDL 
The PDDL (Planning Domain Definition Language)   allows us to express all actions with one action schema, it consists of following four functions:

1. init(self, initial_state, actions, goal_test)  : the constructor creates a knowledge base with initial state, initialises actions and goal_test_func function with goal_test.
2. goal_test(self) : initialises goal_test with kb.
3. act(self, action) :  Performs the action given as argument, along with checks preformed on pre-conditions.

#ACTIONS
Actions are described by a set of action schemas that implicitly define the class Actions:
Actions consists of a precondition(positive and negative) and effect(positive and negative). It consists of following 

Each problem described in chapter 10 has:
	* an initial condition
	* a goal
	* actions with preconditions and effects
		these action are split into _pos (positive conditions) and _neg (negative conditions)
Each problem thus requires a solution which is satisfied by the initial conditions and solved using actions which meet the preconditions and effects.
 

# AIR CARGO PROBLEM from Figure 10.1

The following code defines the initial state for the problem

In [2]:
def air_cargo():
    init = [expr('At(C1, SFO)'),     
            expr('At(C2, JFK)'),
            expr('At(P1, SFO)'),
            expr('At(P2, JFK)'),
            expr('Cargo(C1)'),
            expr('Cargo(C2)'),
            expr('Plane(P1)'),
            expr('Plane(P2)'),
            expr('Airport(JFK)'),
            expr('Airport(SFO)')]

The following code defines the goal_test() function which tests if the solution achieves goal or not. 'required' states the goal:

In [3]:
def goal_test(kb):
        required = [expr('At(C1 , JFK)'), expr('At(C2 ,SFO)')]
        for q in required:
            if kb.ask(q) is False:
                return False
        return True

Actions such as 'load', 'unload'  and fly are deifned with preconditions and effects accompanying them.

In [14]:
# Actions

#  Load
precond_pos = [expr("At(c, a)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"),
               expr("Airport(a)")]
precond_neg = []
effect_add = [expr("In(c, p)")]
effect_rem = [expr("At(c, a)")]
load = Action(expr("Load(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Unload
precond_pos = [expr("In(c, p)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"),
               expr("Airport(a)")]
precond_neg = []
effect_add = [expr("At(c, a)")]
effect_rem = [expr("In(c, p)")]
unload = Action(expr("Unload(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Fly
#  Used 'f' instead of 'from' because 'from' is a python keyword and expr uses eval() function
precond_pos = [expr("At(p, f)"), expr("Plane(p)"), expr("Airport(f)"), expr("Airport(to)")]
precond_neg = []
effect_add = [expr("At(p, to)")]
effect_rem = [expr("At(p, f)")]
fly = Action(expr("Fly(p, f, to)"), [precond_pos, precond_neg], [effect_add, effect_rem])

Finally, the function returns the defined problem using PDLL.

  return PDLL(init, [load, unload, fly], goal_test)

A solution to the air_cargo problem is as follows:

In [2]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")]

We then execute the action on the state's kb.

In [3]:
a = air_cargo()

for action in solution:
    a.act(action)

Now we test whether the actions achieve the goal defined before.

In [4]:
a.goal_test()

True

Yes, the solution is correct. You may try any other solution to check if it achieves the goal or not.

# SPARE TIRE PROBLEM from Figure 10.2:

The problem follows the same structure of function as before, only the predicates are changed.


In [5]:
init = [expr('Tire(Flat)'),
            expr('Tire(Spare)'),
            expr('At(Flat, Axle)'),
            expr('At(Spare, Trunk)')]


required = [expr('At(Spare, Axle)'), expr('At(Flat, Ground)')]

In [6]:
# Actions

# Remove
precond_pos = [expr("At(obj, loc)")]
precond_neg = []
effect_add = [expr("At(obj, Ground)")]
effect_rem = [expr("At(obj, loc)")]
remove = Action(expr("Remove(obj, loc)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# PutOn
precond_pos = [expr("Tire(t)"), expr("At(t, Ground)")]
precond_neg = [expr("At(Flat, Axle)")]
effect_add = [expr("At(t, Axle)")]
effect_rem = [expr("At(t, Ground)")]
put_on = Action(expr("PutOn(t, Axle)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# LeaveOvernight
precond_pos = []
precond_neg = []
effect_add = []
effect_rem = [expr("At(Spare, Ground)"), expr("At(Spare, Axle)"), expr("At(Spare, Trunk)"),
              expr("At(Flat, Ground)"), expr("At(Flat, Axle)"), expr("At(Flat, Trunk)")]
leave_overnight = Action(expr("LeaveOvernight"), [precond_pos, precond_neg],
                         [effect_add, effect_rem])

solution to the spare tire problem from the book is as follows:

In [7]:
solution = [expr("Remove(Flat, Axle)"),
                expr("Remove(Spare, Trunk)"),
                expr("PutOn(Spare, Axle)")]

let's test it in the same way as before :

In [8]:
s = spare_tire()

for action in solution:
    s.act(action)

s.goal_test()

True

The solution achieves the goal.

# THE BLOCKS WORLD from Figure 10.3 :


![pL plot](images/blocks_world.png)

 The above image shows how the transitions between different states can be possible. Watch how the rightmost transition is what we follow in the book.